## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
from calendar import timegm
from datetime import datetime
from time import mktime


## Notes

##### Dates are datetime objects in the excel sheets. I also worked in Epoch time to do the binning. We could change this once we know our preferred method for how we are going to bin it. The First section importing the data includes all sheets but we are only using Pump Data for now. 

## Import the Data

In [2]:
#Get Sheet Dictioanry
#sheets_dict = pd.read_excel('P-16051 Raw Data1.xlsx', sheetname=None)

raw_data_1 = pd.ExcelFile('ChevronDataFiles/P-16051 Raw Data1.xlsx')

In [3]:
vibration = pd.read_excel(raw_data_1,'Vibration')
vibration_filtered = pd.read_excel(raw_data_1,'Vib Filtered')
pump_data = pd.read_excel(raw_data_1,'Pump data')
dataset = pd.read_excel(raw_data_1,'dataset')

In [4]:
type(pump_data['Date'][0])

datetime.datetime

In [4]:
#print("Size of Dataset is: ", dataset.shape)
#dataset.head(3)

In [5]:
#print("Size of Vibration is: ", vibration.shape)
#vibration.head(3)

In [6]:
#print("Size of Vib Filtered is: ", vibration_filtered.shape)
#vibration_filtered.head(3)

In [7]:
print("Size of Pump Data is: ", pump_data.shape)
pump_data.head(3)

Size of Pump Data is:  (7073, 40)


,MBPD,Date,MBPD.1,Date.1,IN/SEC,Date.2,IN/SEC 1,Date.3,%_1,Date.4,...,IN/SEC 3,Date.15,F 3,Date.16,KBPD,Date.17,MBPD.2,Date.18,VIB,Date.19
0,28.85,2012-04-10 09:37:36,2.55,2012-04-10 09:37:36,0.13,2012-04-10 09:37:36,0.06,2012-04-10 09:37:36,41.88,2012-04-10 09:37:36,...,0.02,2012-04-10 09:37:36,104.19,2012-04-10 09:37:36,8.91,2012-04-10 09:37:36,88.65,2012-04-10 09:37:36,0.231,2008-04-21 10:44:00
1,28.87,2012-04-10 09:40:19,2.58,2012-04-10 09:56:14,0.13,2012-04-10 09:48:24,0.05,2012-04-10 09:38:24,42.16,2012-04-10 09:52:14,...,0.01,2012-04-10 09:54:24,104.15,2012-04-10 09:38:09,8.94,2012-04-10 09:38:04,88.81,2012-04-10 09:52:14,0.219,2008-05-12 09:56:00
2,28.79,2012-04-10 09:54:19,2.58,2012-04-10 09:58:14,0.13,2012-04-10 09:57:24,0.07,2012-04-10 09:48:24,42.15,2012-04-10 09:58:14,...,0.01,2012-04-10 09:58:24,105.46,2012-04-10 09:57:09,8.78,2012-04-10 09:40:04,88.63,2012-04-10 09:56:14,0.277,2008-06-09 10:17:00


## Get The Column Names

In [8]:
column_names = pump_data.columns.values

## Epoch Function

In [9]:
def epoch_time(series): 
    epoch_values = []
    pump_data_dropped = series.dropna().reset_index().drop(columns = 'index').to_numpy()

    for i in range(len(pump_data_dropped)):
        try: 
            epoch_values.append(pump_data_dropped[i][0].timestamp())
        
        except AttributeError as error:
            try: 
                epoch_values.append(time.mktime(time.strptime(pump_data_dropped[i][0], "%m/%d/%y %H:%M:%S %p")))
            except TypeError as error_numpy: 
                #print(pump_data_dropped[i][0], type(pump_data_dropped[i][0]))
                epoch_values.append(np.datetime64(pump_data_dropped[i][0], 's').astype(int))

        except IndexError as index_error: 
            break

            
    return epoch_values

## Separate Into Different DataFrames

In [10]:
# Create a clean dataframe
clean_frames = []
counter = 0
for i in range(0, len(column_names), 2): 
    clean_frames.append(pump_data[[column_names[i], column_names[i + 1]]].dropna().reset_index().drop(columns = 'index'))
    clean_frames[counter][column_names[i + 1]] = epoch_time(clean_frames[counter][column_names[i + 1]])
    counter += 1

clean_frame = pd.concat(clean_frames, axis = 1)

In [11]:
clean_frame.head()

,MBPD,Date,MBPD.1,Date.1,IN/SEC,Date.2,IN/SEC 1,Date.3,%_1,Date.4,...,IN/SEC 3,Date.15,F 3,Date.16,KBPD,Date.17,MBPD.2,Date.18,VIB,Date.19
0,28.85,1.334069e+09,2.55,1.334069e+09,0.13,1.334069e+09,0.06,1.334069e+09,41.88,1.334069e+09,...,0.02,1.334069e+09,104.19,1.334069e+09,8.91,1.334069e+09,88.65,1.334069e+09,0.231,1.208775e+09
1,28.87,1.334069e+09,2.58,1.334070e+09,0.13,1.334069e+09,0.05,1.334069e+09,42.16,1.334070e+09,...,0.01,1.334070e+09,104.15,1.334069e+09,8.94,1.334069e+09,88.81,1.334070e+09,0.219,1.210586e+09
2,28.79,1.334070e+09,2.58,1.334070e+09,0.13,1.334070e+09,0.07,1.334069e+09,42.15,1.334070e+09,...,0.01,1.334070e+09,105.46,1.334070e+09,8.78,1.334069e+09,88.63,1.334070e+09,0.277,1.213007e+09
3,28.81,1.334070e+09,2.57,1.334070e+09,0.13,1.334070e+09,0.06,1.334070e+09,42.18,1.334070e+09,...,0.01,1.334070e+09,105.32,1.334070e+09,8.89,1.334070e+09,88.73,1.334070e+09,0.197,1.218105e+09
4,28.83,1.334070e+09,2.57,1.334070e+09,0.13,1.334070e+09,0.06,1.334070e+09,42.05,1.334070e+09,...,0.01,1.334071e+09,104.84,1.334070e+09,8.90,1.334070e+09,88.70,1.334070e+09,0.163,1.222705e+09


In [12]:
# Epoch Times for binning
array_times = []
for epoch in range(1333281600, 1614870337,2628000):
    array_times.append(epoch)
epoch_df = pd.DataFrame(array_times, columns = ['Epoch'])


In [13]:
clean_frame = pd.concat([epoch_df , clean_frame], axis = 1)
clean_frame

,Epoch,MBPD,Date,MBPD.1,Date.1,IN/SEC,Date.2,IN/SEC 1,Date.3,%_1,...,IN/SEC 3,Date.15,F 3,Date.16,KBPD,Date.17,MBPD.2,Date.18,VIB,Date.19
0,1.333282e+09,28.85,1.334069e+09,2.55,1.334069e+09,0.13,1.334069e+09,0.06,1.334069e+09,41.88,...,0.02,1.334069e+09,104.19,1.334069e+09,8.91,1.334069e+09,88.65,1.334069e+09,0.231,1.208775e+09
1,1.335910e+09,28.87,1.334069e+09,2.58,1.334070e+09,0.13,1.334069e+09,0.05,1.334069e+09,42.16,...,0.01,1.334070e+09,104.15,1.334069e+09,8.94,1.334069e+09,88.81,1.334070e+09,0.219,1.210586e+09
2,1.338538e+09,28.79,1.334070e+09,2.58,1.334070e+09,0.13,1.334070e+09,0.07,1.334069e+09,42.15,...,0.01,1.334070e+09,105.46,1.334070e+09,8.78,1.334069e+09,88.63,1.334070e+09,0.277,1.213007e+09
3,1.341166e+09,28.81,1.334070e+09,2.57,1.334070e+09,0.13,1.334070e+09,0.06,1.334070e+09,42.18,...,0.01,1.334070e+09,105.32,1.334070e+09,8.89,1.334070e+09,88.73,1.334070e+09,0.197,1.218105e+09
4,1.343794e+09,28.83,1.334070e+09,2.57,1.334070e+09,0.13,1.334070e+09,0.06,1.334070e+09,42.05,...,0.01,1.334071e+09,104.84,1.334070e+09,8.90,1.334070e+09,88.70,1.334070e+09,0.163,1.222705e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,52.79,1.591451e+09,NaN,NaN
6978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,52.82,1.591451e+09,NaN,NaN
6979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,52.85,1.591452e+09,NaN,NaN
6980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,52.77,1.591452e+09,NaN,NaN


In [14]:
# Bin Them!!

# Get all the values from Epoch
for i in range(len(clean_frame['Epoch']) - 1):
    # Iterate over columns
    for j in range(0, len(column_names), 2): 
        clean_frame[column_names[j]][i] = clean_frame.loc[((clean_frame[column_names[j + 1]] > clean_frame['Epoch'][i]) & (clean_frame[column_names[j +1]] < clean_frame['Epoch'][i + 1])), column_names[j]].mean()


In [15]:
#clean_frame.loc[((clean_frame['Date'] > clean_frame['Epoch'][0]) & (clean_frame['Date'] < clean_frame['Epoch'][1])), 'MBPD.1'].mean()
clean_frame['Epoch'] = pd.to_datetime(clean_frame['Epoch'],unit='s')
clean_frame = clean_frame.loc[:, ~clean_frame.columns.str.startswith('Date')]
clean_frame.head()

,Epoch,MBPD,MBPD.1,IN/SEC,IN/SEC 1,%_1,F 1,API,F 2,PSIG 1,...,RPM 2,%_2,%_3,PSIG 2,IN/SEC 2,IN/SEC 3,F 3,KBPD,MBPD.2,VIB
0,2012-04-01 12:00:00,30.293668,2.465056,0.125286,0.054742,41.322664,642.617802,-3.689418,601.655679,19.087628,...,3352.166667,49.514181,44.622638,19.087628,0.164556,0.018336,107.777573,9.307597,88.229832,0.086
1,2012-05-01 22:00:00,26.361649,2.609066,0.134535,0.056763,40.752648,648.620963,-3.623025,599.771152,18.923879,...,3810.862361,49.305879,44.926353,18.923879,0.165113,0.018863,102.819720,8.291025,84.639348,0.102
2,2012-06-01 08:00:00,25.271181,2.225562,0.139745,0.068258,37.288255,639.751695,-4.415422,590.731442,18.423481,...,1608.770499,48.663606,44.946560,18.423481,0.163456,0.020345,100.497613,5.527477,75.210778,0.107
3,2012-07-01 18:00:00,31.870631,2.129614,0.120030,0.049910,50.544059,643.471935,-4.746252,604.062835,18.440350,...,3950.443103,47.170915,44.673432,18.440350,0.161114,0.018664,106.849654,7.803907,88.117752,0.080
4,2012-08-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.129


In [16]:
clean_frame.to_excel("ChevronDataFiles/P-16051_Raw_Data1_Cleaned.xlsx", sheet_name = 'PumpData')

## Bin The Data to Vizualize

In [33]:
def standard_date_types(series): 
    standard_times = []
    pump_data_dropped = series.dropna().reset_index().drop(columns = 'index').to_numpy()
    for i in range(len(pump_data_dropped)):
        try: 
            standard_times.append(np.datetime64(pump_data_dropped[i][0]))
            
        except ValueError as error:
            try: 
                standard_times.append(np.datetime64(datetime.fromtimestamp(mktime(time.strptime(pump_data_dropped[i][0], "%m/%d/%y %H:%M:%S %p")))))
            except TypeError as error_numpy: 
                standard_times.append((pump_data_dropped[i][0]))
        except IndexError as index_error: 
            break
    return standard_times

In [44]:
MBPD = pump_data[['MBPD', 'Date']].dropna().copy()
MBPD = MBPD.reset_index().drop(columns = ['index'])

In [45]:
MBPD['Date'] = standard_date_types(pump_data['Date'])
MBPD.head()

,MBPD,Date
0,28.85,2012-04-10 09:37:36
1,28.87,2012-04-10 09:40:19
2,28.79,2012-04-10 09:54:19
3,28.81,2012-04-10 09:58:19
4,28.83,2012-04-10 09:59:19


In [56]:
for i in range(len(MBPD)):
    MBPD['Date'][i] = str(MBPD['Date'][i].year) + '-' +  str(MBPD['Date'][i].month)

AttributeError: 'str' object has no attribute 'year'

In [66]:
counts = dict(MBPD['Date'].value_counts())
counts

{'2012-4': 777,
 '2012-7': 776,
 '2020-5': 771,
 '2020-4': 767,
 '2020-6': 763,
 '2012-6': 762,
 '2012-5': 758,
 '2019-11': 743,
 '2019-12': 732}

In [74]:
VIB = pump_data[['VIB', 'Date.19']].dropna().copy()
VIB = VIB.reset_index().drop(columns = ['index'])
VIB['Date.19'] = standard_date_types(pump_data['Date.19'])
VIB.head()

,VIB,Date.19
0,0.231,2008-04-21 10:44:00
1,0.219,2008-05-12 09:56:00
2,0.277,2008-06-09 10:17:00
3,0.197,2008-08-07 10:26:00
4,0.163,2008-09-29 16:17:00


In [75]:
for i in range(len(VIB)):
    VIB['Date.19'][i] = str(VIB['Date.19'][i].year) + '-' +  str(VIB['Date.19'][i].month)
    
counts = dict(VIB['Date.19'].value_counts())
counts

<ipython-input-75-e02f73b7aca8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VIB['Date.19'][i] = str(VIB['Date.19'][i].year) + '-' +  str(VIB['Date.19'][i].month)
C:\Users\rodri\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


{'2020-6': 13,
 '2018-7': 12,
 '2017-9': 12,
 '2020-5': 11,
 '2020-4': 11,
 '2018-4': 10,
 '2019-10': 8,
 '2010-8': 8,
 '2010-7': 7,
 '2018-9': 5,
 '2020-2': 4,
 '2019-12': 4,
 '2018-6': 4,
 '2010-12': 4,
 '2018-8': 4,
 '2010-9': 4,
 '2009-12': 3,
 '2020-3': 3,
 '2010-4': 3,
 '2016-5': 3,
 '2019-8': 3,
 '2009-10': 2,
 '2013-7': 2,
 '2016-1': 2,
 '2017-7': 2,
 '2009-4': 2,
 '2013-8': 2,
 '2017-6': 2,
 '2017-8': 2,
 '2011-5': 1,
 '2017-5': 1,
 '2015-12': 1,
 '2013-4': 1,
 '2009-9': 1,
 '2011-11': 1,
 '2011-6': 1,
 '2009-5': 1,
 '2012-5': 1,
 '2014-4': 1,
 '2012-3': 1,
 '2017-12': 1,
 '2012-6': 1,
 '2015-3': 1,
 '2011-3': 1,
 '2008-9': 1,
 '2019-1': 1,
 '2008-11': 1,
 '2016-9': 1,
 '2015-1': 1,
 '2019-9': 1,
 '2009-2': 1,
 '2013-2': 1,
 '2015-9': 1,
 '2009-8': 1,
 '2008-6': 1,
 '2011-8': 1,
 '2018-11': 1,
 '2008-4': 1,
 '2019-5': 1,
 '2012-8': 1,
 '2012-11': 1,
 '2017-1': 1,
 '2011-9': 1,
 '2019-7': 1,
 '2015-2': 1,
 '2011-7': 1,
 '2012-2': 1,
 '2015-7': 1,
 '2012-12': 1,
 '2013-5': 1,
 '